In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load data columns we want to use

In [2]:
games_all = pd.read_csv('data/games_detailed_info.csv', 
                    engine='python',
                    encoding='utf-8',
                    usecols=['id', 'thumbnail', 'primary', 'yearpublished', 
                             'minplayers', 'maxplayers', 'playingtime', 'minage', 
                             'boardgamecategory', 'boardgamemechanic', 
                             'boardgamefamily', 'boardgamedesigner', 'usersrated', 
                             'average', 'Board Game Rank', 'Strategy Game Rank', 'Family Game Rank', 
                             'averageweight', 'Party Game Rank', 'Abstract Game Rank', 'Thematic Rank', 
                             'War Game Rank', 'Customizable Rank', "Children's Game Rank"])

# Step 2: rename columns to better names

In [3]:
games_all = games_all.rename(columns={'primary': 'name',
                              'yearpublished': 'year',
                              'minplayers': 'min_players',
                              'maxplayers': 'max_players',
                              'playingtime': 'playing_time',
                              'minage': 'min_age',
                              'boardgamecategory': 'category',
                              'boardgamemechanic': 'mechanic',
                              'boardgamefamily': 'family',
                              'boardgamedesigner': 'designer',
                              'usersrated': 'users_rated',        
                              'average': 'rating',
                              'Board Game Rank': 'rank',
                              'averageweight': 'complexity',
                              'Strategy Game Rank': 'type_strategy',
                              'Family Game Rank': 'type_family',
                              'Party Game Rank': 'type_party',
                              'Abstract Game Rank': 'type_abstract',
                              'Thematic Rank': 'type_thematic',
                              'War Game Rank': 'type_war',
                              'Customizable Rank': 'type_customizable',
                              "Children's Game Rank": 'type_children'})

# Step 3: Convert columns to better data formats
- arrays stored as string --> arrays of strings (boardgamecategory, boardgamefamily, boardgamemechanic, boardgamedesigner)
- numbers stored as strings --> numbers (Board Game Rank)
- type ranks --> booleans

- method to reassign columns in a loop suggested [here](https://github.com/pandas-dev/pandas/issues/29435)

In [4]:
def convert_array_col(cell):
  if type(cell) != str:
    return []
  else:
    clean_step_1 = cell.strip('[]')
    clean_step_2 = re.sub(r"\s/\s", "/", clean_step_1)
    split = re.split(r", ['\"]", clean_step_2)
    clean_step_3 = [re.sub(r"^'|'$|^\"|\"$", "", c) for c in split]
    return clean_step_3

def convert_str_to_int(cell):
    return np.nan if cell == 'Not Ranked' else float(cell)

def convert_rank_to_bool(cell):
  if cell == 'Not Ranked':
    return 0
  else:
    return 0 if pd.isna(float(cell)) else 1

In [5]:
array_col_to_convert = ['category', 'mechanic', 'family', 'designer']
str_col_to_convert = ['rank']
rank_col_to_convert = ['type_strategy', 'type_family', 'type_party', 'type_abstract', 
                       'type_thematic', 'type_war', 'type_customizable', 'type_children']

games_all = games_all.assign(**{col: games_all[col].apply(convert_array_col) for col in array_col_to_convert})
games_all = games_all.assign(**{col: games_all[col].apply(convert_str_to_int) for col in str_col_to_convert})
games_all = games_all.assign(**{col: games_all[col].apply(convert_rank_to_bool) for col in rank_col_to_convert})

# Step 4: Filter to top 1000 games with type


In [6]:
types = rank_col_to_convert
games = games_all.loc[games_all[types].max(axis=1) == 1].nsmallest(n=1000, columns='rank').reset_index(drop=True)

# Step 5: Process theme data
- add themes from family for games without themes
- combine Pirates and Nautical (to keep # of themes per cluster minimal)
- add War to themes for Wargames
- separate out themes from categories (as per the hierarchy for categories describe [here](https://boardgamegeek.com/wiki/page/Category))

In [7]:
category_hierarchy = {
#     "activities": ['City Building', 'Civilization', 'Educational', 'Puzzle', 'Racing', 'Territory Building',
#                    'Transportation'],
#     "components": ['Card Game', 'Collectible Components', 'Dice', 'Electronic', 'Miniatures', 'Print & Play'],
#     "Non-Game": ['Expansion for Base-game'],
#     "Party Games": ['Party Game'],
#     "Abstract Strategy Games": ['Abstract Strategy'],
#     "Age": ["Children's Game", 'Mature/Adult'],
    "wargames": ['American Civil War', 'American Indian Wars', 'American Revolutionary War', 'Civil War',
                 'Modern Warfare', 'Napoleonic', 'Pike and Shot', 'Post-Napoleonic', 'Vietnam War',
                 'World War I', 'World War II'],
#     "skills": ['Action/Dexterity', 'Bluffing', 'Deduction', 'Economic', 'Math', 'Maze', 'Memory', 'Number',
#                'Negotiation', 'Real-time', 'Trivia', 'Word Game'],
    "themes": ['Adventure', 'Age of Reason', 'American West', 'Ancient', 'Animals', 'Arabian', 'Aviation/Flight',
               'Comic Book/Strip', 'Environmental', 'Exploration', 'Fantasy', 'Farming', 'Fighting', 'Horror',
               'Humor', 'Industry/Manufacturing', 'Mafia', 'Medical', 'Medieval', 'Movies/TV/Radio theme',
               'Murder/Mystery', 'Mythology',  'Nautical', 'Novel-based', 'Pirates', 'Political', 'Prehistoric',
               'Racing', 'Religious', 'Renaissance', 'Science Fiction', 'Space Exploration', 'Spies/Secret Agents',
               'Sports', 'Trains', 'Transportation', 'Travel', 'Video Game Theme', 'Zombies', 'Food & Drink']
}

for big_c in category_hierarchy.keys():
  games[big_c] = games.category.apply(lambda x: [c for c in category_hierarchy[big_c] if c in x])

In [8]:
games.themes = games.apply(lambda row: row['themes'] + ['War'] if len(row['wargames']) > 0 else row['themes'], axis=1)
games = games.drop('wargames', axis=1)

In [9]:
def add_theme_from_family(row):
    if len(row.themes) > 0: #only do conversion if there are no themes already
        return row.themes
    else:
        themes = []
        families_as_string = " ".join(row.family)
        ## existing themes
        themes.append('Ancient') if any([f in families_as_string for f in ['Ancient:']]) else None
        themes.append('Prehistoric') if 'Paleontology' in families_as_string else None
        themes.append('Animals') if 'Animals:' in families_as_string else None
        themes.append('Novel-based') if any([f in families_as_string for f in ['Authors:', 'Books:']]) else None
        themes.append('Comic Book/Strip') if any([f in families_as_string for f in ['Comic Books:', 'Anime', 'Superheroes']]) else None
        themes.append('Science Fiction') if any([f in families_as_string for f in ['Aliens', 'Cyberpunk', 'Robots', 'Sci-Fi', 'Time Travel', 'UFOs', 'Post-Apolcalyptic', 'Steampunk']]) else None
        themes.append('Fantasy') if any([f in families_as_string for f in ['Demons', 'Dragons', 'Dwarves', 'Fairies', 'Goblins', 'Trolls', 'Unicorns','Druids']]) else None
        themes.append('Horror') if any([f in families_as_string for f in ['Ghosts', 'Monsters', 'Vampires', 'Werewolves', 'Nightmares', 'Spooky', 'Witches']]) else None
        themes.append('Mythology') if any([f in families_as_string for f in ['Minotaurs', 'Mythology:']]) else None
        themes.append('Zombies') if 'Zombies' in families_as_string else None
        themes.append('Medical') if any([f in families_as_string for f in ['Medical:', 'Medic', 'Biology']]) else None
        themes.append('Movies/TV/Radio theme') if any([f in families_as_string for f in ['Movies:', 'TV Shows:', 'Movie', 'Television']]) else None
        themes.append('Exploration') if '/Explorer' in families_as_string else None
        themes.append('Farming') if 'Farmer' in families_as_string else None
        themes.append('Industry/Manufacturing') if any([f in families_as_string for f in ['Merchant', 'Mining', 'Petroleum']]) else None
        themes.append('Political') if 'Political:' in families_as_string else None
        themes.append('Religious') if any([f in families_as_string for f in ['Religious:', 'Temple']]) else None
        themes.append('Space Exploration') if any([f in families_as_string for f in ['Space', 'Astronomy']]) else None
        themes.append('Sports') if 'Sports' in families_as_string else None
        themes.append('Video Game Theme') if 'Video Game Theme' in families_as_string else None
        themes.append('Aviation/Flight') if 'Airships' in families_as_string else None
        themes.append('Transportation') if any([f in families_as_string for f in ['Automotive', 'Motorcycles', 'Submarines', 'Trucks']]) else None
        themes.append('Humor') if 'Circus' in families_as_string else None
        themes.append('Environmental') if any([f in families_as_string for f in ['Climate Change', 'Earthquakes', 'Volcanoes', 'Weather']]) else None
        themes.append('Murder/Mystery') if any([f in families_as_string for f in ['Mystery']]) else None
        themes.append('Fighting') if any([f in families_as_string for f in ['Gladiators', 'Ninjas', 'Samurai', 'Survival']]) else None
        themes.append('Mafia') if 'Jail' in families_as_string else None
        themes.append('Medieval') if 'Camelot' in families_as_string else None
        themes.append('War') if 'Warfare' in families_as_string else None
        ## new themes
        themes.append('Food & Drink') if any([f in families_as_string for f in ['Food & Drink:', 'Food/', 'Mushrooms', 'Bistros']]) else None
        themes.append('Parks, Forests & Beaches') if any([f in families_as_string for f in ['Parks', 'Beaches', 'Deserts', 'Flowers', 'Gardening', 'Trees','Tropical', 'Under The Sea']]) else None
        themes.append('City Life') if any([f in families_as_string for f in [': City', 'Canals', 'Construction', 'Firefighting']]) else None
        return themes

games.themes = games.apply(lambda row: add_theme_from_family(row), axis=1)

In [10]:
# amalgamates Pirates into Nautical to avoid repetition
def change_pirate(row):
    themes = row.themes.copy()
    themes.remove('Pirates')
    if 'Nautical' not in themes:
        themes.append('Nautical')
    return themes

games.themes = games.apply(lambda row: change_pirate(row) if 'Pirates' in row['themes'] else row['themes'], axis=1)

In [11]:
# changes names of some themes for conciseness
changes = {
    'Parks, Forests & Beaches': 'Parks & Beaches',
    'Murder/Mystery': 'Murder & Mystery', 
    'Spies/Secret Agents': 'Spies', 
    'Aviation/Flight': 'Planes', 
    'Industry/Manufacturing': 'Industry', 
    'Movies/TV/Radio theme': 'Movies & TV', 
    'Video Game Theme': 'Video Games', 
    'Comic Book/Strip': 'Comic Books'
}
def change_names(row):
    return [changes[t] if t in changes.keys() else t for t in row.themes]
    
games.themes = games.apply(lambda row: change_names(row), axis=1)

# Step 6: Create columns of theme clusters
- these clusters are created by us
- drop theme column once we have allocated themes to the theme clusters

In [12]:
big_themes = {
    'history': ['Age of Reason', 'American West', 'Ancient', 'Renaissance', 'Prehistoric', 'Medieval', 'Arabian'],
    'nature': ['Animals', 'Environmental', 'Farming', 'Fantasy', 'Mythology', 'Religious', 'Parks & Beaches'],
    'journey': ['Planes', 'Trains', 'Transportation', 'Travel', 'Exploration', 'Nautical', 'Adventure'],
    'culture': ['Comic Books', 'Novel-based', 'Video Games', 'Movies & TV', 'Humor', 'Sports', 'Food & Drink'],
    'science': ['Science Fiction', 'Space Exploration', 'Industry', 'Medical', 'Zombies', 'Horror', 'City Life'],
    'war': ['Fighting', 'Mafia', 'Murder & Mystery', 'Racing', 'Spies', 'Political', 'War'],
}

In [13]:
for big_t in big_themes.keys():
  games['theme_cluster_' + big_t] = games.themes.apply(lambda x: [t for t in big_themes[big_t] if t in x])

In [14]:
games = games.drop('themes', axis=1)

# Step 7: Create links
- base similarity measure on having same categories, families, mechanics, designers, types
- MultiLabelBinarizer makes one column per item in each of the above categories (1 present, 0 not) [as suggested [here](https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list)]
- cosine_similarity calculates the similarity score based on the vectors above
- filter to > 0.4 because we consider this "similar"
- create a list of links for each game of format {source: gameID, target: otherGameId, weight: similarity_score}
- then drop category, family, mechanic, and designer columns, since they are no longer needed

In [15]:
data_for_links = games[['category', 'family', 'mechanic', 'designer',
                        'type_strategy', 'type_family', 'type_party', 'type_abstract',
                        'type_thematic', 'type_war', 'type_customizable', 'type_children']]

In [16]:
mlb = MultiLabelBinarizer()
data_for_links = data_for_links.join(pd.DataFrame(mlb.fit_transform(data_for_links.category), columns=[f'category_{c}' for c in mlb.classes_]))
data_for_links = data_for_links.join(pd.DataFrame(mlb.fit_transform(data_for_links.mechanic), columns=[f'mechanic_{c}' for c in mlb.classes_]))
data_for_links = data_for_links.join(pd.DataFrame(mlb.fit_transform(data_for_links.family), columns=[f'family_{c}' for c in mlb.classes_]))
data_for_links = data_for_links.join(pd.DataFrame(mlb.fit_transform(data_for_links.designer), columns=[f'designer_{c}' for c in mlb.classes_]))

In [17]:
data_for_links = data_for_links.drop(['category', 'family', 'mechanic', 'designer'], axis=1)
games = games.drop(['category', 'family', 'mechanic', 'designer'], axis=1)

In [18]:
links = pd.DataFrame(cosine_similarity(data_for_links, Y=None, dense_output=True))

In [19]:
links = links.applymap(lambda cell: 0.0 if cell < 0.4 else cell)

In [20]:
links = links.rename(index=games.id, columns=games.id)

In [21]:
links_aggregated = {}
for row in links.index:
    game_links = []
    for col in links.columns:
        if links[row][col] > 0 and row != col:
            game_links.append({'source': row, 'target': col, 'weight': links[row][col]})
    links_aggregated[row] = game_links

In [22]:
games['links'] = games.id.apply(lambda id: links_aggregated[id])

# Step 8: Create link to BGG website for each game

In [23]:
games['url'] = games.id.apply(lambda id: f'https://boardgamegeek.com/boardgame/{id}')

# SAVE TO FILE

In [24]:
games.to_json('data/games_clean.json', orient='records', double_precision=4)